In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import cv2

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Convolution2D, Flatten, MaxPooling2D, Dropout
from keras.utils import np_utils

from skimage.feature import hog
from skimage import data, exposure
import pickle

Using TensorFlow backend.


In [2]:
!wget https://www.floydhub.com/api/v1/resources/xRu8AHanSHPfwpN2AAsdjb/X.pickle?content=true&rename=xpickle
!mv X.pickle\?content\=true X.pickle

--2018-10-29 15:19:41--  https://www.floydhub.com/api/v1/resources/xRu8AHanSHPfwpN2AAsdjb/X.pickle?content=true
Resolving www.floydhub.com (www.floydhub.com)... 104.25.128.32, 104.25.129.32, 2606:4700:20::6819:8120, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.25.128.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘X.pickle?content=true’

X.pickle?content=tr     [          <=>       ] 574.28M  13.2MB/s    in 46s     

2018-10-29 15:20:28 (12.5 MB/s) - ‘X.pickle?content=true’ saved [602173604]



In [0]:
!wget https://www.floydhub.com/api/v1/resources/xRu8AHanSHPfwpN2AAsdjb/y.pickle?content=true&rename=ypickle
!mv y.pickle\?content\=true y.pickle

In [0]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [0]:
y[y == '+'] = 10
y[y=='-'] = 11
y[y=='times'] = 12
y[y=='forward_slash'] = 13


In [0]:
y = np_utils.to_categorical(y)

In [0]:
print(X.shape,y.shape)

In [0]:
 from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,shuffle=True,random_state=42)

In [0]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

In [0]:
model = Sequential()

model.add(Dense(800, input_shape=(960,)))
model.add(Activation('relu'))


model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(14))
model.add(Activation('softmax'))
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
from tensorflow.keras.callbacks import TensorBoard
import time
import tensorflow as tf

In [0]:
NAME = 'CROHME-digits-gist-split-10e-{}'.format(int(time.time()))

In [0]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [0]:
# keras.backend.get_session().run(tf.initialize_all_variables())

In [0]:
hist = model.fit(X_train, y_train,
         epochs=10,
         shuffle=True,
         batch_size=512,
         validation_data=(X_test,y_test),callbacks = [tensorboard])

In [0]:
import h5py
file_name = 'gist-crohme-digits-10e-features.h5'
model.save(file_name)  # creates a HDF5 file 'my_model.h5'
f = h5py.File(file_name, 'r+')
del f['optimizer_weights']
f.close()